# mysql中关于超时设置

MySQL中有两个关于连接超时的配置项: `wait_timeout`和`interactive_timeout`。

他们之间在某些条件下会互相继承.

### wait_timeout

```
The number of seconds the server waits for activity on a noninteractive connection before closing it. Before MySQL 5.1.41, this timeout applies only to TCP/IP connections, not to connections made through Unix socket files, named pipes, or shared memory. On thread startup, the session wait_timeout value is initialized from the global wait_timeout value or from the global interactive_timeout value, depending on the type of client (as defined by the CLIENT_INTERACTIVE connect option to mysql_real_connect()). See also interactive_timeout.
```

### interactive_timeout

```
The number of seconds the server waits for activity on an interactive connection before closing it. An interactive client is defined as a client that uses the CLIENT_INTERACTIVE option to mysql_real_connect(). See alsowait_timeout.
```

简单的说 interactive就是交互式的终端，例如在shell里面直接执行mysql，出现形如mysql>的提示符后就是交互式的连接。

而`mysql -e ‘select xx from xx where xx’` 这样的直接返回结果的方式就是非交互式的连接。

### 参数继承关系

1. 过socket连接 timeout会从哪个global timeout继承?

    通过socket登录，timeout 继承于global.interactive_timeout.
    
2. 通过TCP/IP client 连接， timeout会从哪个global timeout继承?
    
    通过TCP/IP client 连接后的wait_timeout 仍然继承于 global.interactive_timeout.

### 起效关系

1. timeout值，对于处于运行状态SQL语句是否起效（即是否等价于执行超时）？
    
    正在执行状态的等待时间不计入timeout时间。即SQL运行再久也不会因为timeout的配置而中断.
    
2. 同一个session中，wait_timeout 和 interacitve_timeout是否都会生效?

    只有wait_timeout 会真正起到超时限制的作用.
    
3. global timeout和session timeout是否都会作为超时判断依据？
    
    只有session级别 timeout 会起作用。即一个session开始后，无论如何修改global级别的timeout都不会影响该session.

### 总结

1. 超时时间只对非活动状态的connection进行计算。
2. 超时时间只以session级别的wait_timeout 为超时依据，global级别只决定session初始化时的超时默认值。
3. 交互式连接的wait_timeout 继承于global的interactive_timeout。非交互式连接的wait_timeout继承于global的wait_timeout
4. 继承关系和超时对 TCP/IP 和 Socket 连接均有效果